## Part 1

In [1]:
import platform,os, sys
os_name = platform.system()

In [2]:
import findspark

In [3]:
if os_name == "Windows":
    findspark.init()
    img_dir = "D:/Learn/GitRepos/Spark-The-Definitive-Guide/data/deep-learning-images"
    temp_loc = "D:/temp/"
    master = "local[*]"
else:
    spark_home = "/usr/hdp/current/spark2-client"
    findspark.init(spark_home)
    hdfs_home = "/user/" + os.getenv("HOME").split("/")[2]
    img_dir = hdfs_home+"/dataSets/spark-guide/deep-learning-images/"
    temp_loc = os.getenv("HOME") + "/temp/"
    master = "yarn"
   # sys.path.append(os.path.expanduser('hdfs:///user/kranthidr/pythonpath/sparkdl.zip'))
    sys.path.append(os.path.expanduser('~/pythonpath/sparkdl.zip'))
    sys.path.append(os.path.expanduser('~/pythonpath/tensorframes.zip'))
    sys.path.append(os.path.expanduser('~/pythonpath/tensorflow.zip'))
    sys.path.append(os.path.expanduser("~/.local/lib/python2.7/site-packages"))
    

In [4]:
sys.path
#/home/kranthidr/.local/lib/python2.7/site-packages

['/usr/hdp/current/spark2-client/python',
 '/usr/hdp/current/spark2-client/python/lib/py4j-0.10.6-src.zip',
 '',
 '/usr/lib/python2.7/site-packages/avro-1.7.7-py2.7.egg',
 '/usr/lib64/python27.zip',
 '/usr/lib64/python2.7',
 '/usr/lib64/python2.7/plat-linux2',
 '/usr/lib64/python2.7/lib-tk',
 '/usr/lib64/python2.7/lib-old',
 '/usr/lib64/python2.7/lib-dynload',
 '/home/kranthidr/.local/lib/python2.7/site-packages',
 '/usr/lib64/python2.7/site-packages',
 '/usr/lib64/python2.7/site-packages/gtk-2.0',
 '/usr/lib/python2.7/site-packages',
 '/usr/lib/python2.7/site-packages/IPython/extensions',
 '/home/kranthidr/.ipython',
 '/home/kranthidr/pythonpath/sparkdl.zip',
 '/home/kranthidr/pythonpath/tensorframes.zip',
 '/home/kranthidr/pythonpath/tensorflow.zip',
 '/home/kranthidr/.local/lib/python2.7/site-packages']

In [5]:
findspark.find()

'/usr/hdp/current/spark2-client'

In [6]:
img_dir

'/user/kranthidr/dataSets/spark-guide/deep-learning-images/'

In [7]:
temp_loc

'/home/kranthidr/temp/'

In [8]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]")\
.config("spark.driver.memory", "2g")\
.config("spark.jars.packages","databricks:spark-deep-learning:1.2.0-spark2.3-s_2.11,databricks:tensorframes:0.5.0-s_2.11")\
.appName("ch31_DeepLearningImgTransLearn").getOrCreate()

# https://mvnrepository.com/artifact/org.tensorflow/tensorflow
#'org.tensorflow:tensorflow:jar:1.6.0'

#.config("spark.jars.packages","databricks:spark-deep-learning:1.2.0-spark2.3-s_2.11,databricks:tensorframes:0.5.0-s_2.11")\
# .config("spark.driver.memory", "8g")\
# .config("spark.executor.memory", "3g")\
# .config("spark.executor.cores", "4")\
# .config("spark.executor.instances", "10")\


#!/usr/bin/env bash

# This file is sourced when running various Spark programs.
# Copy it as spark-env.sh and edit that to configure Spark for your site.

# Options read in YARN client mode
#SPARK_EXECUTOR_INSTANCES="2" #Number of workers to start (Default: 2)
#SPARK_EXECUTOR_CORES="1" #Number of cores for the workers (Default: 1).
#SPARK_EXECUTOR_MEMORY="1G" #Memory per Worker (e.g. 1000M, 2G) (Default: 1G)
#SPARK_DRIVER_MEMORY="512M" #Memory for Master (e.g. 1000M, 2G) (Default: 512 Mb)
#SPARK_YARN_APP_NAME="spark" #The name of your application (Default: Spark)
#SPARK_YARN_QUEUE="default" #The hadoop queue to use for allocation requests (Default: default)
#SPARK_YARN_DIST_FILES="" #Comma separated list of files to be distributed with the job.
#SPARK_YARN_DIST_ARCHIVES="" #Comma separated list of archives to be distributed with the job.

In [9]:
sc = spark.sparkContext

In [10]:
for x in sc._conf.getAll():
    if "/proxy/" in x[1]:
        print(x[1])

In [11]:
spark

In [12]:
from sparkdl.image import imageIO
from pyspark.ml.image import ImageSchema

/home/kranthidr/.local/lib/python2.7/site-packages/pandas/core/nanops.py:39: UserWarning: The installed version of bottleneck 0.7.0 is not supported in pandas and will be not be used
The minimum supported version is 1.0.0

  ver=ver, min_ver=_MIN_BOTTLENECK_VERSION), UserWarning)
Using TensorFlow backend.


In [13]:
from pyspark.sql.functions import lit
from pyspark.sql.functions import expr
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import udf

In [14]:
tulips_df = ImageSchema.readImages(img_dir + "/tulips").withColumn("label", lit(1))
tulips_df.count()

# Signature: ImageSchema.readImages(path, recursive=False, 
#                                   numPartitions=-1, dropImageFailures=False, 
#                                   sampleRatio=1.0, seed=0)
# Docstring:
# Reads the directory of images from the local or remote source.

# .. note:: If multiple jobs are run in parallel with different sampleRatio or recursive flag,
#     there may be a race condition where one job overwrites the hadoop configs of another.

# .. note:: If sample ratio is less than 1, sampling uses a PathFilter that is efficient but
#     potentially non-deterministic.

# :param str path: Path to the image directory.
# :param bool recursive: Recursive search flag.
# :param int numPartitions: Number of DataFrame partitions.
# :param bool dropImageFailures: Drop the files that are not valid images.
# :param float sampleRatio: Fraction of the images loaded.
# :param int seed: Random number seed.
# :return: a :class:`DataFrame` with a single column of "images",
#        see ImageSchema for details.

30

In [15]:
# In[16]:
daisy_df = imageIO.readImagesWithCustomFn(img_dir + "/daisy", decode_f=imageIO.PIL_decode).withColumn("label", lit(0))
daisy_df.count()

# Signature: imageIO.readImagesWithCustomFn(path, decode_f, numPartition=None)
# Docstring:
# Read a directory of images (or a single image) into a DataFrame using a custom library to
# decode the images.

# :param path: str, file path.
# :param decode_f: function to decode the raw bytes into an array compatible with one of the
#     supported OpenCv modes. see @imageIO.PIL_decode for an example.

30

In [16]:
# In[ ]:
tulips_train, tulips_test = tulips_df.randomSplit([0.8, 0.2])
daisy_train, daisy_test = daisy_df.randomSplit([0.8, 0.2])

In [17]:
# In[ ]:
train_images_df = tulips_train.unionAll(daisy_train).cache()
test_images_df = tulips_test.unionAll(daisy_test).cache()

In [18]:
print(train_images_df.count())
print(test_images_df.count())

47
13


In [19]:
train_images_df.printSchema()

root
 |-- image: struct (nullable = true)
 |    |-- origin: string (nullable = true)
 |    |-- height: integer (nullable = false)
 |    |-- width: integer (nullable = false)
 |    |-- nChannels: integer (nullable = false)
 |    |-- mode: integer (nullable = false)
 |    |-- data: binary (nullable = false)
 |-- label: integer (nullable = false)



In [20]:
print(train_images_df.schema)

StructType(List(StructField(image,StructType(List(StructField(origin,StringType,true),StructField(height,IntegerType,false),StructField(width,IntegerType,false),StructField(nChannels,IntegerType,false),StructField(mode,IntegerType,false),StructField(data,BinaryType,false))),true),StructField(label,IntegerType,false)))


## Part 2 

In [21]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline

In [22]:
from sparkdl import DeepImageFeaturizer

In [23]:
featurizer = DeepImageFeaturizer(inputCol="image", outputCol="features", modelName="InceptionV3")

# Init signature: DeepImageFeaturizer(inputCol=None, outputCol=None, 
#                                     modelName=None, 
#                                     scaleHint='SCALE_AREA_AVERAGING')
# Docstring:     
# Applies the model specified by its popular name, with its prediction layer(s) chopped off,
# to the image column in DataFrame. The output is a MLlib Vector so that DeepImageFeaturizer
# can be used in a MLlib Pipeline.
# The input image column should be ImageSchema.

In [24]:
lr = LogisticRegression(maxIter=20, regParam=0.05, elasticNetParam=0.3, labelCol="label")

p = Pipeline(stages=[featurizer, lr])

model = p.fit(train_images_df)    # train_images_df is a dataset of images and labels

In [25]:
# Inspect training error
fitted_train_df = model.transform(train_images_df.limit(10))

In [26]:
fitted_train_df.printSchema()

root
 |-- image: struct (nullable = true)
 |    |-- origin: string (nullable = true)
 |    |-- height: integer (nullable = false)
 |    |-- width: integer (nullable = false)
 |    |-- nChannels: integer (nullable = false)
 |    |-- mode: integer (nullable = false)
 |    |-- data: binary (nullable = false)
 |-- label: integer (nullable = false)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [27]:
predictionAndLabels = fitted_train_df.select("prediction", "label")

In [28]:
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
print("Training set accuracy = " + str(evaluator.evaluate(predictionAndLabels)))

Training set accuracy = 1.0


In [29]:
# Inspect testing error
fitted_test_df = model.transform(test_images_df.limit(10))
predictionAndLabels = fitted_test_df.select("prediction", "label")

evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
print("Test set accuracy = " + str(evaluator.evaluate(predictionAndLabels)))

Test set accuracy = 1.0


In [30]:
fitted_test_df.printSchema()

root
 |-- image: struct (nullable = true)
 |    |-- origin: string (nullable = true)
 |    |-- height: integer (nullable = false)
 |    |-- width: integer (nullable = false)
 |    |-- nChannels: integer (nullable = false)
 |    |-- mode: integer (nullable = false)
 |    |-- data: binary (nullable = false)
 |-- label: integer (nullable = false)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [31]:
fitted_test_df.select("probability").take(1)

[Row(probability=DenseVector([0.0174, 0.9826]))]

In [32]:
fitted_test_df.select("probability").take(1)[0][0][1]

0.9826166501737116

In [33]:
# COMMAND ----------
def _p11(v):
  return float(v[1])

p11 = udf(_p11, DoubleType())

In [34]:
# COMMAND ----------
def _p1(v):
  return float(v.array[1])

p1 = udf(_p1, DoubleType())

In [35]:
fitted_test_df.withColumn("p_1", p11(fitted_test_df.probability)).show(5)

+--------------------+-----+--------------------+--------------------+--------------------+----------+------------------+
|               image|label|            features|       rawPrediction|         probability|prediction|               p_1|
+--------------------+-----+--------------------+--------------------+--------------------+----------+------------------+
|[hdfs://nn01.itve...|    1|[0.0,0.4272569119...|[-4.0347062230499...|[0.01738334982628...|       1.0|0.9826166501737116|
|[hdfs://nn01.itve...|    1|[0.0,0.0,0.0,0.0,...|[-3.9517044147499...|[0.01885939804503...|       1.0|0.9811406019549624|
|[hdfs://nn01.itve...|    1|[0.0,0.0,2.751705...|[-3.6991019136515...|[0.02414817581778...|       1.0| 0.975851824182212|
|[hdfs://nn01.itve...|    1|[0.0,0.3870728909...|[-2.1245066400553...|[0.10673762407765...|       1.0| 0.893262375922349|
|[hdfs://nn01.itve...|    1|[0.0,0.0,2.664251...|[-3.7499874029297...|[0.02297765270852...|       1.0|0.9770223472914745|
+--------------------+--

In [36]:
df = fitted_test_df.withColumn("p_1", p1(fitted_test_df.probability))

In [37]:
df.show(10)

+--------------------+-----+--------------------+--------------------+--------------------+----------+--------------------+
|               image|label|            features|       rawPrediction|         probability|prediction|                 p_1|
+--------------------+-----+--------------------+--------------------+--------------------+----------+--------------------+
|[hdfs://nn01.itve...|    1|[0.0,0.4272569119...|[-4.0347062230499...|[0.01738334982628...|       1.0|  0.9826166501737116|
|[hdfs://nn01.itve...|    1|[0.0,0.0,0.0,0.0,...|[-3.9517044147499...|[0.01885939804503...|       1.0|  0.9811406019549624|
|[hdfs://nn01.itve...|    1|[0.0,0.0,2.751705...|[-3.6991019136515...|[0.02414817581778...|       1.0|   0.975851824182212|
|[hdfs://nn01.itve...|    1|[0.0,0.3870728909...|[-2.1245066400553...|[0.10673762407765...|       1.0|   0.893262375922349|
|[hdfs://nn01.itve...|    1|[0.0,0.0,2.664251...|[-3.7499874029297...|[0.02297765270852...|       1.0|  0.9770223472914745|
|[hdfs:/

In [38]:
wrong_df = df.orderBy(expr("abs(p_1 - label)"), ascending=False)

In [39]:
wrong_df.show(10)

+--------------------+-----+--------------------+--------------------+--------------------+----------+--------------------+
|               image|label|            features|       rawPrediction|         probability|prediction|                 p_1|
+--------------------+-----+--------------------+--------------------+--------------------+----------+--------------------+
|[hdfs://nn01.itve...|    0|[0.0,0.5201152563...|[0.65368643126472...|[0.65784071003978...|       0.0| 0.34215928996021183|
|[hdfs://nn01.itve...|    1|[0.0,0.3870728909...|[-2.1245066400553...|[0.10673762407765...|       1.0|   0.893262375922349|
|[hdfs://nn01.itve...|    0|[0.60087662935256...|[2.50445830281958...|[0.92445377317046...|       0.0| 0.07554622682953908|
|[hdfs://nn01.itve...|    1|[0.05437384545803...|[-3.0934721893845...|[0.04337732525674...|       1.0|  0.9566226747432591|
|[hdfs://nn01.itve...|    1|[0.0,0.0,2.751705...|[-3.6991019136515...|[0.02414817581778...|       1.0|   0.975851824182212|
|[hdfs:/

In [40]:
# COMMAND ----------
# img_dir_comb = "D:\Learn\GitRepos\Spark-The-Definitive-Guide\data\deep-learning-images\Combined30"
# image1_df = imageIO.readImagesWithCustomFn(img_dir_comb, decode_f=imageIO.PIL_decode)
# image1_df.count() 
#Will not recurse through directories

In [41]:
train_images_df.show(10)

+--------------------+-----+
|               image|label|
+--------------------+-----+
|[hdfs://nn01.itve...|    1|
|[hdfs://nn01.itve...|    1|
|[hdfs://nn01.itve...|    1|
|[hdfs://nn01.itve...|    1|
|[hdfs://nn01.itve...|    1|
|[hdfs://nn01.itve...|    1|
|[hdfs://nn01.itve...|    1|
|[hdfs://nn01.itve...|    1|
|[hdfs://nn01.itve...|    1|
|[hdfs://nn01.itve...|    1|
+--------------------+-----+
only showing top 10 rows



In [42]:
train_images_df.show(10)

+--------------------+-----+
|               image|label|
+--------------------+-----+
|[hdfs://nn01.itve...|    1|
|[hdfs://nn01.itve...|    1|
|[hdfs://nn01.itve...|    1|
|[hdfs://nn01.itve...|    1|
|[hdfs://nn01.itve...|    1|
|[hdfs://nn01.itve...|    1|
|[hdfs://nn01.itve...|    1|
|[hdfs://nn01.itve...|    1|
|[hdfs://nn01.itve...|    1|
|[hdfs://nn01.itve...|    1|
+--------------------+-----+
only showing top 10 rows



In [43]:
image_df = train_images_df.unionAll(test_images_df)

In [44]:
image_df.show(10)

+--------------------+-----+
|               image|label|
+--------------------+-----+
|[hdfs://nn01.itve...|    1|
|[hdfs://nn01.itve...|    1|
|[hdfs://nn01.itve...|    1|
|[hdfs://nn01.itve...|    1|
|[hdfs://nn01.itve...|    1|
|[hdfs://nn01.itve...|    1|
|[hdfs://nn01.itve...|    1|
|[hdfs://nn01.itve...|    1|
|[hdfs://nn01.itve...|    1|
|[hdfs://nn01.itve...|    1|
+--------------------+-----+
only showing top 10 rows



In [45]:
predictions_transfer_df = model.transform(image_df).cache()

In [46]:
predictions_transfer_df.show(5)

+--------------------+-----+--------------------+--------------------+--------------------+----------+
|               image|label|            features|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+--------------------+----------+
|[hdfs://nn01.itve...|    1|[0.0,0.0,0.0,0.0,...|[-3.2850161604833...|[0.03608881550804...|       1.0|
|[hdfs://nn01.itve...|    1|[0.0,0.9765790700...|[-4.2179093221610...|[0.01451560059519...|       1.0|
|[hdfs://nn01.itve...|    1|[1.43618118762969...|[-3.2848202070215...|[0.03609563264576...|       1.0|
|[hdfs://nn01.itve...|    1|[1.04818153381347...|[-3.3858371871075...|[0.03274103154254...|       1.0|
|[hdfs://nn01.itve...|    1|[0.0,0.0,0.0,0.0,...|[-3.7595377075086...|[0.02276422536008...|       1.0|
+--------------------+-----+--------------------+--------------------+--------------------+----------+
only showing top 5 rows

